In [1]:
import tensorflow as tf
import numpy as np 
import torch
import time

In [2]:
def nearest_neighbor(matrix):
    w = len(matrix)
    h = len(matrix[0])
    neighbors = [[0 for x in range(w)] for y in range(h)]
    for i in range (0, w):
        for j in range (0, h):
            entry = {
                "left": matrix[i][(j-1)%w],
                "right": matrix[i][(j+1)%h],
                "up": matrix[(i-1)%w][j],
                "down": matrix[(i+1)%w][j]
            }
            neighbors[i][j] = entry
    return neighbors        
    
    

In [3]:
matrix = np.arange(25).reshape((5,5))
print(matrix)
neighbors = nearest_neighbor(matrix)
print(neighbors)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[{'left': 4, 'right': 1, 'up': 20, 'down': 5}, {'left': 0, 'right': 2, 'up': 21, 'down': 6}, {'left': 1, 'right': 3, 'up': 22, 'down': 7}, {'left': 2, 'right': 4, 'up': 23, 'down': 8}, {'left': 3, 'right': 0, 'up': 24, 'down': 9}], [{'left': 9, 'right': 6, 'up': 0, 'down': 10}, {'left': 5, 'right': 7, 'up': 1, 'down': 11}, {'left': 6, 'right': 8, 'up': 2, 'down': 12}, {'left': 7, 'right': 9, 'up': 3, 'down': 13}, {'left': 8, 'right': 5, 'up': 4, 'down': 14}], [{'left': 14, 'right': 11, 'up': 5, 'down': 15}, {'left': 10, 'right': 12, 'up': 6, 'down': 16}, {'left': 11, 'right': 13, 'up': 7, 'down': 17}, {'left': 12, 'right': 14, 'up': 8, 'down': 18}, {'left': 13, 'right': 10, 'up': 9, 'down': 19}], [{'left': 19, 'right': 16, 'up': 10, 'down': 20}, {'left': 15, 'right': 17, 'up': 11, 'down': 21}, {'left': 16, 'right': 18, 'up': 12, 'down': 22}, {'left': 17, 'right': 19, 'up': 13, 'down': 23}, {'lef

In [4]:
def higher_dim_nearest_neighbor(tensor):
    w = len(matrix)
    h = len(matrix[0])
    d = len(matrix[0][0])
    neighbors = [[[0 for x in range(w)] for y in range(h)] for z in range(d)]
    for i in range (0, w):
        for j in range (0, h):
            for k in range (0,d):
                entry = [6]
                entry[0] = matrix[i][(j-1)%w][k],
                entry[1] = matrix[i][(j+1)%h][k],
                entry[2] = matrix[(i-1)%w][j][k],
                entry[3] = matrix[(i+1)%w][j][k],
                entry[4] = matrix[i][j][(k-1)%d],
                entry[5] = matrix[i][j][(k+1)%d]
            neighbors[i][j][z] = entry
    return neighbors   

In [2]:
### Contracting Tensors

def indices ( num_indices , _range ):
    _list = [[]]
    return ( _index_gen_helper ( num_indices , _range , _list ))
def _index_gen_helper ( num_indices , _range , _list ):
    _ret = []

    for i in _list :
        for j in range ( _range ):
            _temp = i. copy ()
            _temp . extend ([ j ])
            _ret . append ( _temp )
    if num_indices == 1:
        return _ret
    else :
        return _index_gen_helper ( num_indices -1 , _range , _ret )
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
num_independent_indices = 8
spin_degree = 2

A=torch.rand((2,2,2,2))
B=torch.rand((2,2,2,2))

start = time.time()
for i ,j ,k ,l ,a ,b ,c , d in indices ( num_independent_indices , spin_degree ):  
    c_1 += A[i][j][a][b]*A[j][i][c][d]*A[k][l][b][a]*A[l][k][d][c]
    c_2 += A[i][j][a][b]*A[j][i][c][d]*A[k][l][b][a]*B[l][k][d][c]
    c_3 += A[i][j][a][b]*A[j][i][c][d]*B[k][l][b][a]*A[l][k][d][c]
    c_4 += A[i][j][a][b]*A[j][i][c][d]*B[k][l][b][a]*B[l][k][d][c]
end = time.time()
    
print(c_1)
print(c_2)
print(c_3)
print(c_4)
print(end-start)

tensor(6.5934)
tensor(7.8423)
tensor(7.8423)
tensor(10.2817)
0.07076883316040039


In [9]:
AB= [A,B]
count=0
C_i = [0 for x in range(2**4)]
D_i = [0 for x in range(2**4)]
start=time.time()
for i ,j ,k ,l in indices(4,2):    
    C_i[count]=torch.einsum('ijab, jicd, klba, lkdc', AB[i], AB[j], AB[k], AB[l])
    #X = np.einsum('ijab, jicd', AB[i], AB[j])
    #Y = np.einsum('ijab, jicd', AB[k], AB[l])
    #D_i[count] = np.einsum('abcd, badc', X, Y)
    count+=1
end=time.time()
print(C_i)
#print(D_i)
print(end-start)


[tensor(15.7748), tensor(16.8209), tensor(16.8209), tensor(18.0626), tensor(16.8209), tensor(18.6648), tensor(17.7975), tensor(19.9884), tensor(16.8209), tensor(17.7975), tensor(18.6648), tensor(19.9884), tensor(18.0626), tensor(19.9884), tensor(19.9884), tensor(22.3388)]
0.004233121871948242


In [10]:

def instant_indices(num_independent_indices, inds, i):
    for j in range (0, num_independent_indices):
        inds[j] = i%2
        i//=2
    return inds

In [11]:
inds=[0 for x in range(32)]
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
start = time.time()
for i in range (2**8):
    inds = instant_indices(num_independent_indices, inds, i)
    c_1 += A[inds[0]][inds[1]][inds[4]][inds[5]]*A[inds[1]][inds[0]][inds[6]][inds[7]]*A[inds[2]][inds[3]][inds[5]][inds[4]]*A[inds[3]][inds[2]][inds[7]][inds[6]]
    c_2 += A[inds[0]][inds[1]][inds[4]][inds[5]]*A[inds[1]][inds[0]][inds[6]][inds[7]]*A[inds[2]][inds[3]][inds[5]][inds[4]]*B[inds[3]][inds[2]][inds[7]][inds[6]]
    c_3 += A[inds[0]][inds[1]][inds[4]][inds[5]]*A[inds[1]][inds[0]][inds[6]][inds[7]]*B[inds[2]][inds[3]][inds[5]][inds[4]]*A[inds[3]][inds[2]][inds[7]][inds[6]]
    c_4 += A[inds[0]][inds[1]][inds[4]][inds[5]]*A[inds[1]][inds[0]][inds[6]][inds[7]]*B[inds[2]][inds[3]][inds[5]][inds[4]]*B[inds[3]][inds[2]][inds[7]][inds[6]]
end = time.time()    
print(c_1)
print(c_2)
print(c_3)
print(c_4) 
print(end-start)

tensor(15.7748)
tensor(16.8209)
tensor(16.8209)
tensor(18.0626)
0.07987594604492188


In [3]:
A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))


In [9]:
print(A)
print(B)

c_1=0
c_2=0
c_3=0
c_4=0
c_5=0
c_6=0
c_7=0
c_8=0
start=time.time()
for  a ,b ,c , d, e, f, g, h, i, j ,k ,l ,m ,n, o, p, q, r in indices ( 18 , 2 ):  
    c_1 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*A[g][h][k][l]*A[i][g][n][o]*A[h][i][q][r]
    c_2 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*A[g][h][k][l]*A[i][g][n][o]*B[h][i][q][r]
    c_3 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*A[g][h][k][l]*B[i][g][n][o]*A[h][i][q][r]
    c_4 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*A[g][h][k][l]*B[i][g][n][o]*B[h][i][q][r]
    c_5 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*B[g][h][k][l]*A[i][g][n][o]*A[h][i][q][r]
    c_6 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*B[g][h][k][l]*A[i][g][n][o]*B[h][i][q][r]
    c_7 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*B[g][h][k][l]*B[i][g][n][o]*A[h][i][q][r]
    c_8 += A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*B[g][h][k][l]*B[i][g][n][o]*B[h][i][q][r]
    
end=time.time()   
print(c_1) 
print(c_2)
print(c_3)
print(c_4)
print(c_5)
print(c_6)
print(c_7)
print(c_8)
print(end-start)

tensor([[[[0.2927, 0.6908],
          [0.0614, 0.2820]],

         [[0.7225, 0.3655],
          [0.2412, 0.3700]]],


        [[[0.3025, 0.4392],
          [0.7169, 0.2018]],

         [[0.0522, 0.3946],
          [0.7652, 0.3757]]]])
tensor([[[[0.2281, 0.9448],
          [0.2913, 0.9326]],

         [[0.4149, 0.0527],
          [0.3584, 0.5231]]],


        [[[0.0349, 0.2980],
          [0.8537, 0.3870]],

         [[0.9053, 0.1372],
          [0.9033, 0.6677]]]])
tensor(57.4416)
tensor(73.2549)
tensor(73.2554)
tensor(92.5636)
tensor(73.2557)
tensor(92.5636)
tensor(92.5630)
tensor(120.8514)
328.5142123699188


In [7]:
C_i = [0 for x in range(2**9)]
D_i = [0 for x in range(2**9)]
AB=[A,B]

start=time.time()
count=0
for i ,j ,k ,l ,m ,n ,o ,p ,q in indices(9, 2):    
    X = torch.einsum('abjk, camn->bcjkmn', AB[i], AB[j])
    X = torch.einsum('bcjkmn, bcpq->jkmnpq', X, AB[k])
    X = torch.einsum('jkmnpq, delj->deklmnpq', X, AB[l])
    X = torch.einsum('deklmnpq, fdom->efklnopq', X, AB[m])
    X = torch.einsum('efklnopq, efrp->klnoqr', X, AB[n])
    X = torch.einsum('klnoqr, ghkl->ghnoqr', X, AB[o])
    X = torch.einsum('ghnoqr, igno->hiqr', X, AB[p])
    X = torch.einsum('hiqr,hiqr', X, AB[q])
    D_i[count]=torch.einsum('abjk, camn, bcpq, delj, fdom, efrp, ghkl, igno, hiqr', AB[i], AB[j], AB[k], AB[l], AB[m], AB[n], AB[o], AB[p], AB[q])
    C_i[count] = X
    count+=1
end=time.time()
for i in range (8):
    print(C_i[i])
    print(D_i[i])

print(end-start)
#A[a][b][j][k]*A[c][a][m][n]*A[b][c][p][q]*A[d][e][l][j]*A[f][d][o][m]*A[e][f][r][p]*A[g][h][k][l]*A[i][g][n][o]*A[h][i][q][r]

tensor(57.4467)
tensor(57.4467)
tensor(73.2667)
tensor(73.2667)
tensor(73.2667)
tensor(73.2667)
tensor(92.5773)
tensor(92.5773)
tensor(73.2667)
tensor(73.2667)
tensor(92.5773)
tensor(92.5773)
tensor(92.5773)
tensor(92.5773)
tensor(120.8674)
tensor(120.8674)
0.8574471473693848


In [31]:
netA = [0] * 16
netB = [0] * 16
for i in range (16):
    netA[i]=torch.rand((2,2,2,2))
    netB[i]=torch.rand((2,2,2,2))

C_i = [0 for x in range(2**16)]

AB=[netA,netB]

start=time.time()
count=0
for a, b, c, d, e, f, g, h, i ,j ,k ,l ,m ,n ,o ,p in indices(16, 2):    
    X = torch.einsum('abeh, cail, dcmp, bdqt-> ehilmpqt', AB[a][0], AB[b][1], AB[c][2], AB[d][3])
    Y = torch.einsum('abfe, caji, dcnm, bdrq-> fejinmrq', AB[e][4], AB[f][5], AB[g][6], AB[h][7])
    Z = torch.einsum('abgf, cakj, dcon, bdsr-> gfkjonsr', AB[i][8], AB[j][9], AB[k][10], AB[l][11])
    W = torch.einsum('abhg, calk, dcpo, bdts-> hglkpots', AB[m][12], AB[n][13], AB[o][14], AB[p][15])
    C_i[count] = torch.einsum('ehilmpqt, fejinmrq, gfkjonsr, hglkpots', X, Y, Z, W)
    count+=1
end=time.time()
for i in range (20):
    print(C_i[i])


print(end-start)

tensor(90311.6953)
tensor(90171.0781)
tensor(78384.7266)
tensor(78678.1719)
tensor(79363.8281)
tensor(79021.4922)
tensor(67868.3438)
tensor(68256.0859)
tensor(91298.8125)
tensor(94390.8516)
tensor(79354.9844)
tensor(82527.)
tensor(77564.4062)
tensor(80416.9531)
tensor(66495.1094)
tensor(69731.2656)
tensor(136501.5000)
tensor(133692.)
tensor(118763.6016)
tensor(116899.8438)
90.59476113319397


In [6]:
netA = [[0 for i in range (5)]]*5
netB = [[0 for i in range (5)]]*5
for i in range (5):
    for j in range (5):
        netA[i][j]=torch.rand((2,2,2,2))
        netB[i][j]=torch.rand((2,2,2,2))

C_i = [0 for x in range(2**25)]

AB=[netA,netB]

start=time.time()
count=0
for a, b, c, d, e, f, g, h, i ,j ,k, l  in indices(12, 2):    
    X = torch.einsum('abfg, cahi, dcjk, edlm, beno -> fghijklmno', AB[a][0][0], AB[b][0][1], AB[c][0][2], AB[d][0][3], AB[e][0][4]) 
    Y = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[f][1][0], AB[g][1][1], AB[h][1][2], AB[i][1][3], AB[j][1][4])
    XY = torch.einsum('fghijklmno, pfqhrjsltn -> gpiqkrmson', X, Y)
    Z = torch.einsum('abfp, cahq, dcjr, edls, bent -> fphqjrlsnt' , AB[k][2][0], AB[l][2][1], AB[0][2][2], AB[0][2][3], AB[0][2][4])
    XYZ = torch.einsum('gpiqkrmson, fphqjrlsnt -> gfihkjmlon', XY, Z)
    U = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[0][3][0], AB[0][3][1], AB[0][3][2], AB[0][3][3], AB[0][3][4])
    XYZU = torch.einsum('gfihkjmlon, pfqhrjsltn -> gpiqkrmsot', XYZ, U)
    V = torch.einsum('abgp, caiq, dckr, edms, beot -> gpiqkrmsot', AB[0][4][0], AB[0][4][1], AB[0][4][2], AB[0][4][3], AB[0][4][4])
    C_i[count] = torch.einsum('gpiqkrmsot, gpiqkrmsot', XYZU, V)
    count+=1
end=time.time()
for i in range (0,20):
    print(C_i[i])


print(end-start)

tensor(10875745.)
tensor(10003504.)
tensor(15160137.)
tensor(12806679.)
tensor(11625867.)
tensor(10688977.)
tensor(16158760.)
tensor(13675444.)
tensor(10127021.)
tensor(9315468.)
tensor(14113564.)
tensor(11925300.)
tensor(11691054.)
tensor(10745720.)
tensor(16246996.)
tensor(13746946.)
tensor(10454817.)
tensor(9487443.)
tensor(14614929.)
tensor(12139886.)
10.403019905090332


In [4]:
netA = [[0 for i in range (6)]]*6
netB = [[0 for i in range (6)]]*6

for i in range (6):
    for j in range (6):
        netA[i][j]=torch.rand((2,2,2,2))
        netB[i][j]=torch.rand((2,2,2,2))

C_i = [0 for x in range(4096)]

AB=[netA,netB]

start=time.time()
count=0

for a, b, c, d, e, f, g, h, i ,j ,k, l  in indices(12, 2):    
    X = torch.einsum('abnh, caoi, dcpj, edqk, ferl, bfsm -> nhoipjqkrlsm', AB[a][0][0], AB[b][0][1], AB[c][0][2], AB[d][0][3], AB[e][0][4], AB[f][0][5]) 
    Y = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[g][1][0], AB[h][1][1], AB[i][1][2], AB[j][1][3], AB[k][1][4], AB[l][1][5])
    XY = torch.einsum('nhoipjqkrlsm, tnuovpwqxrys -> htiujvkwlxmy', X, Y)
    Z = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[0][2][0], AB[0][2][1], AB[0][2][2], AB[0][2][3], AB[0][2][4], AB[0][2][5])
    XYZ = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XY, Z)
    U = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[0][3][0], AB[0][3][1], AB[0][3][2], AB[0][3][3], AB[0][3][4], AB[0][3][5])
    XYZU = torch.einsum('hniojpkqlrms, tnuovpwqxrys -> htiujvkwlxmy', XYZ, U)
    V = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[0][4][0], AB[0][4][1], AB[0][4][2], AB[0][4][3], AB[0][4][4], AB[0][4][5])
    XYZUV = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XYZU, V)
    W = torch.einsum('abhn, caio, dcjp, edkq, felr, bfms -> hniojpkqlrms', AB[0][5][0], AB[0][5][1], AB[0][5][2], AB[0][5][3], AB[0][5][4], AB[0][5][5])
    C_i[count] = torch.einsum('hniojpkqlrms, hniojpkqlrms', XYZUV, W)
    count+=1
end=time.time()
for i in range (20):
    print(C_i[i])


print(end-start)

tensor(2.5179e+12)
tensor(2.6282e+12)
tensor(1.3279e+12)
tensor(1.3274e+12)
tensor(2.0377e+12)
tensor(2.0948e+12)
tensor(1.0506e+12)
tensor(1.0372e+12)
tensor(2.2114e+12)
tensor(2.3082e+12)
tensor(1.1662e+12)
tensor(1.1657e+12)
tensor(1.8500e+12)
tensor(1.9036e+12)
tensor(9.5513e+11)
tensor(9.4367e+11)
tensor(2.1481e+12)
tensor(2.2421e+12)
tensor(1.1327e+12)
tensor(1.1323e+12)
24.319133758544922
